# Primer on Decorators 
https://realpython.com/primer-on-python-decorators/#classes-as-decorators

- Functions are **first-class objects** in Python: they can be passed aroung and used as arguments.
- **Inner functions**: functions defined inside other functions.
- **Decorators** wrap a function, modifying its behavior. A decorator is also a regular Python function.

## Decorators and Arguments

In [94]:
def do_twice(func):
    # After using the decorator, our new function will be the wrapped part
    # We don't need to pass func as an argument to the wrapper
    # func is available on the function scope now.
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

# When we make the call of the wrapped function, we will be basically
# doing wrapper_do_twice(*args, **kwargs) and this function will take
# care of passing the argumnets to the original func.

In [95]:
@do_twice
def say_whee():
    print("Whee!")

In [96]:
@do_twice
def greet(name):
    print(f"Hello {name}")

In [97]:
say_whee()

Whee!
Whee!


In [98]:
greet('World')

Hello World
Hello World


## Returning Values

In [99]:
def do_twice(func):
    # After using the decorator, our new function will be the wrapped part
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

In [100]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

In [101]:
return_greeting("Adam")

Creating greeting
Creating greeting


'Hi Adam'

After decorating, the signature of the "final" function is not very helpful anymore (technically accurate, but somewhat confusing):

In [102]:
return_greeting

<function __main__.do_twice.<locals>.wrapper_do_twice(*args, **kwargs)>

## @functools.wraps:

In [104]:
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

In [105]:
@do_twice
def say_whee():
    print("Whee!")

In [109]:
print(say_whee)
print(say_whee.__name__)

<function say_whee at 0x00000172E74858C8>
say_whee


## Decorator boilerplate

In [110]:
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator

Decorators don't have to wrap the functions they are decorating. In the example below, we don't need to use wraps, since the decorator returns the original function:

In [113]:
import random
PLUGINS = dict()

def register(func):
    """Register a function as a plug-in"""
    PLUGINS[func.__name__] = func
    return func

@register
def say_hello(name):
    return f"Hello {name}"

@register
def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

## Decorators and Class Methods

In [7]:
class Circle:
    def __init__(self, radius):
        self._radius = radius
        
    # We have to be explicit about the setter method
    # and we can also use the same name as the original method
    @property
    def radius(self):
        """Get value of radius"""
        return self._radius

    @radius.setter
    def radius(self, value):
        """Set radius, raise error if negative"""
        if value >= 0:
            self._radius = value
        else:
            raise ValueError("Radius must be positive")
    # For this one, we won't be allowed to set are
    @property
    def area(self):
        """Calculate area inside circle"""
        return self.pi() * self.radius**2

    def cylinder_volume(self, height):
        """Calculate volume of cylinder with circle as base"""
        return self.area * height

    @classmethod
    def unit_circle(cls):
        """Factory method creating a circle with radius 1"""
        return cls(1)

    @staticmethod
    def pi():
        """Value of π, could use math.pi instead though"""
        return 3.1415926535

In [8]:
circlea = Circle(10)

In [9]:
circlea.area

314.15926535

## @property syntax

In [ ]:
class Values:
    def __init__(self):
        self._value1 = 0
        self._value2 = 0
        self._value3 = 0
        self._value4 = 0
        self._value5 = 0

    @property
    def value1(self):
        return self._value1

    @value1.setter
    def value1(self, value):
        self._value1 = value if value % 2 == 0 else 0

## Nesting Decorators

In [11]:
import functools

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")           # 4
        return value
    return wrapper_debug

def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

Decorators are executed on the order they are listed. In this case, do_twice is execute on line 11 from the cell above (basically calling debug(do_twice(greet()))   ).

In [12]:
@debug
@do_twice
def greet(name):
    print(f"Hello {name}")

In [13]:
greet("Diego")

Calling greet('Diego')
Hello Diego
Hello Diego
'greet' returned None


## Decorators with Arguments

In [26]:
def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat

In [27]:
@repeat(10)
def greet(name):
    print(f"Hello {name}")

In [28]:
greet("John")

Hello John
Hello John
Hello John
Hello John
Hello John
Hello John
Hello John
Hello John
Hello John
Hello John


## Decorator Objects

In [34]:
RETRIES_LIMIT = 3

class WithRetry:

    def __init__(self, retries_limit=RETRIES_LIMIT, allowed_exceptions=None):
        self.retries_limit = retries_limit
        self.allowed_exceptions = allowed_exceptions or (ControlledException,)

    def __call__(self, operation):

        @wraps(operation)
        def wrapped(*args, **kwargs):
            last_raised = None

            for _ in range(self.retries_limit):
                try:
                    return operation(*args, **kwargs)
                except self.allowed_exceptions as e:
                    logger.info("retrying %s due to %s", operation, e)
                    last_raised = e
            raise last_raised

        return wrapped

In [33]:
@WithRetry(retries_limit=5)
def run_with_custom_retries_limit(task):
    return task.run()

# Descriptors

The first way to use a descritor is by writing a class with methods from the descriptor protocol:

In [118]:
class ClassDescriptor:
    
    def __get__(self, instance, owner):
        print("Accessing the attribute to get the value")
        return 42
    
    # Recommended way to implement read-only descriptors
    def __set__(self, instance, value):
        print("Accessing the attribute to set the value")
        raise AttributeError("Cannot change the value")
        
class Foo:
    descriptor = ClassDescriptor()

In [119]:
Foo().descriptor

Accessing the attribute to get the value
<class '__main__.Foo'>


42

In [5]:
Foo().descriptor = 100

Accessing the attribute to set the value


AttributeError: Cannot change the value

An equivalent way is by using the function property:

In [17]:
class FooProperty:
    
    def getter(self) -> object:
        print("accessing the attribute to get the value")
        return 42

    def setter(self, value) -> None:
        print("accessing the attribute to set the value")
        raise AttributeError("Cannot change the value")
    
    # property returns a property object that implements
    # the description protocol
    attribute = property(getter, setter)

In [15]:
FooProperty().attribute

accessing the attribute to get the value


42

Finally, we can use decorators:

In [26]:
class FooDecorator:
    
    @property
    def attribute(self) -> object:
        print("accessing the attribute to get the value")
        return 42
    
    # If not specified, the setter will not allow setting
    # the attribute
    @attribute.setter
    def attribute(self, value) -> None:
        print("accessing the attribute to set the value")
        raise AttributeError("Cannot change the value")

In [21]:
FooDecorator().attribute

accessing the attribute to get the value


42

## Python Descriptors in Methods and Functions

All functions are non-data descriptors which return bound methods when they are invoked from an object. In pure Python:

In [75]:
class Function(object):
    ...
    def __get__(self, obj, objtype=None):
        "Simulate func_descr_get() in Objects/funcobject.c"
        if obj is None:
            return self
        return types.MethodType(self, obj)

As an example, let's define a class with a function:

In [76]:
class D(object):
    def f(self, x):
        return x
    
d = D()

In [82]:
# Access through the class dictionary does not invoke __get__ and
# it just returns the original function
print(D.__dict__['f'])

<function D.f at 0x000002E554FD8048>


In [89]:
# Dotted access from a class calls __get__, but obj is None
# so it also returns the original function
print(D.f)

<function D.f at 0x000002E554FD8048>


In [88]:
# Dotted access from an instance calls __get__, but this time it returns
# the function wrapped in the bound method object
print(d.f)

<bound method D.f of <__main__.D object at 0x000002E554FAE518>>


The bound method has access to the function, to the instance and to the class internally:

In [87]:
print(d.f.__func__)
print(d.f.__self__)
print(d.f.__class__)

<function D.f at 0x000002E554FD8048>
<class 'method'>


## How Attributes Are Accessed with the Lookup Chain

In [94]:
class Vehicle():
    can_fly = False
    number_of_weels = 0

class Car(Vehicle):
    number_of_weels = 4

    def __init__(self, color):
        self.color = color

In [96]:
my_car = Car("red")
# Instance attributes and methods
print(my_car.__dict__)
# Class attributes and methods. We can also call type(my_car).__dict__
print(Car.__dict__)

{'color': 'red'}
{'__module__': '__main__', 'number_of_weels': 4, '__init__': <function Car.__init__ at 0x000002E554FD8D90>, '__doc__': None}


In [104]:
Car.__base__

__main__.Vehicle

How about accessing the objects. This use:

In [111]:
print(my_car.color)
print(my_car.number_of_weels)
print(my_car.can_fly)

red
4
False


Is equivalent to doing this:

In [112]:
print(my_car.__dict__['color'])
print(Car.__dict__['number_of_weels'])
print(Car.__base__.__dict__['can_fly'])

red
4
False


**Method Resolution Order**  
- Results from \_\_get\_\_ from the *data descriptor* named after the attribute you are looking for
- Value from object's \_\_dict\_\_ for the key named after the attribute
- Results from \_\_get\_\_ from the *non-data descriptor* named after the attribute you are looking for
- Value from object type's \_\_dict\_\_ for the key named after the attribute
- Value from object parent type's \_\_dict\_\_ for the key named after the attribute
- Previous step is repeated for all the parent's types following method resolution order for the object
- AttributeError

## How to Use Python Descriptors Properly

In [ ]:
__get__(self, obj, type=None) -> object
__set__(self, obj, value) -> None

Keep these in mind:  
- **self** is the instance of the descritor you're writing
- **obj** is the instance of the object your descriptor is attached to
- **type** is the type of the object the descriptor is attached to

For \_\_set\_\_, you don't have the type variable (we can only call .\_\_set\_\_ on the object, not on the class).

Example illustrating the variables:

In [121]:
class ClassDescriptor:
    
    def __get__(self, instance, owner):
        print(self)
        print(instance)
        print(owner)
    
    # Recommended way to implement read-only descriptors
    def __set__(self, instance, value):
        print("Accessing the attribute to set the value")
        raise AttributeError("Cannot change the value")
        
class Foo:
    descriptor = ClassDescriptor()

In [122]:
Foo().descriptor

<class '__main__.Foo'>


Do not store values on the descriptor (these would be shared among different objects as a class attribute). Instead, this is the correct implementation:

In [203]:
class OneDigitNumericValue():
    # So you don't need to name every instances of the descriptor manually
    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, obj, type=None) -> object:
        return obj.__dict__.get(self.name) or 0

    def __set__(self, obj, value) -> None:
        obj.__dict__[self.name] = value
        

class Foo():
    number = OneDigitNumericValue()

my_foo_object = Foo()
my_second_foo_object = Foo()

my_foo_object.number = 3
print(my_foo_object.number)
print(my_second_foo_object.number)

my_third_foo_object = Foo()
print(my_third_foo_object.number)

3
0
0


## Why Use Python Descriptors

**D.R.Y. Code**  
Instead of doing this:

In [204]:
class Values:
    def __init__(self):
        self._value1 = 0
        self._value2 = 0
        self._value3 = 0
        self._value4 = 0
        self._value5 = 0

    @property
    def value1(self):
        return self._value1

    @value1.setter
    def value1(self, value):
        self._value1 = value if value % 2 == 0 else 0

    @property
    def value2(self):
        return self._value2

    @value2.setter
    def value2(self, value):
        self._value2 = value if value % 2 == 0 else 0

Do this:

In [205]:
class EvenNumber:
    
    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, obj, type=None) -> object:
        return obj.__dict__.get(self.name) or 0

    def __set__(self, obj, value) -> None:
        obj.__dict__[self.name] = (value if value % 2 == 0 else 0)

In [206]:
class Values:
    value1 =EvenNumber()
    value2 =EvenNumber()

# Abstract Base Classes

These exist to be inherited, but never instantiated. Python provides the abc module to define abstract base classes. 
- ABC tells that you cannot instantiate from that class
- @abstractmethod tells that you must override .calcuate_payroll() when inheriting from Employee

In [207]:
from abc import ABC, abstractmethod

class Employee(ABC):
    def __init__(self, id, name):
        self.id = id
        self.name = name

    @abstractmethod
    def calculate_payroll(self):
        pass

In [215]:
class EmployeeSpecific(Employee):
    def __init__(self, id, name):
        self.id = id
        self.name = name
        
    def calculate_payroll(self):
        print("Overriding the original method")

In [216]:
EmployeeSpecific(10, 'John')